### 0. 프레임워크 로드, 함수정의

In [2]:
import os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# 모델학습,검증,시각화 등을 위한 함수 정의

# 전체 데이터셋 히트맵으로 시각화 
def draw_corrmap(df):
  plt.figure(figsize=(10, 8))
  y_corrmat = df.corr()
  mask = np.zeros_like(y_corrmat, dtype=np.bool) # 절반 mask
  mask[np.triu_indices_from(mask)] = True
  sns.heatmap(y_corrmat, #데이터
              annot = True, #셀값 표기 유무
              mask = mask,
              annot_kws = {"size": 10}, #셀값 글자크기
              cmap = 'PuBu', #히트맵 색
              cbar = True, #colorbar 유무
              linewidth=3,
              cbar_kws={"shrink": .5},
              fmt='.2f') # 선굵기
  
# boxplot, scatterplot -> 2col기준 출력
def boxplot(x, y, data):
    sns.boxplot(x=x, y=y, data=data)
    plt.title(f"{y} distribution by {x} / Box Plot")
    plt.legend()
    plt.show()

def scatterplot(x, y, data):
    plt.scatter(x=x, y=y, data=data)
    plt.title(f"{y} distribution by {x} / scatter Plot")
    plt.legend()
    plt.show()

### 1. 기계적특성치/온도데이터 로드

In [ ]:
path = "C:\\Users\\Chaaaaabbo\\Desktop\\3DP\\230201_신규데이터진행"

# 기계적특성치 데이터 로드
mechanic_path = os.path.join(path,  'mechanical_property_all.csv')
mechanic_df = pd.read_csv(mechanic_path, index_col=0)
print(mechanic_df)

# 온도 데이터 로드
temperature_path = os.path.join(path, 'temperature_data_143ea_desc.csv')
temperature_df = pd.read_csv(temperature_path, index_col=0)
print(temperature_df)


In [ ]:
mechanic_df = mechanic_df.rename(columns = {'Number':'item_no'})
mechanic_df.describe()


In [ ]:
temperature_df.describe()

In [ ]:
## 온도/기계적 데이터 통합
merge_data = pd.merge(temperature_df,mechanic_df, how='inner', on='item_no')
print('없는 시편 => ',set(temperature_df['item_no'].unique()) - set(mechanic_df['item_no'].unique()))
# >> ()

In [ ]:
print(merge_data.describe())

In [14]:
to_corr_list = ['item_no', 'mean', 'median', 'max', 'image_layer',
                'power(w)', 'speed(mm/sec)', 'angle', 'yielding(MPa)', 
                'tensile(MPa)','density(g/cm^3)', 'specimen_position', 'energy_density',]

In [ ]:
# 필요 col으로 재정의
merge_data_to_corr = pd.DataFrame(merge_data, columns=to_corr_list)
merge_data_to_corr

In [16]:
merge_data_to_corr = merge_data_to_corr.rename(columns = {'mean':'temperature_mean', 'median' : 'temperature_median'})

#### 기계적 특성 / 온도 특성 상관관계 확인

In [ ]:
draw_corrmap(merge_data_to_corr)

In [ ]:
scatterplot(x="temperature_mean", y="energy_density", data=merge_data_to_corr)

In [ ]:
scatterplot(x="energy_density", y="temperature_mean", data=merge_data_to_corr)

In [ ]:
sns.pairplot(merge_data_to_corr, palette="vlag",
             vars=['temperature_mean', 'power(w)', 'speed(mm/sec)', 'angle', 'energy_density'],
                hue = 'specimen_position')

In [ ]:
sns.pairplot(merge_data_to_corr, palette="vlag",
             vars=['temperature_mean', 'power(w)', 'speed(mm/sec)',
                'tensile(MPa)','density(g/cm^3)', 'energy_density','specimen_position'] ,
                hue = 'angle')

### 온도 - Area


### 1) 0도
---

In [ ]:
path = "C:\\Users\\Chaaaaabbo\\Desktop\\3DP\\230201_신규데이터진행\\"
files = os.listdir(path)

list_of_df = []
for file in os.listdir(path):
    file_path = os.path.join(path, file)
    df_temp = pd.read_csv(file_path)
    list_of_df.append(df_temp)

image_df = pd.concat(list_of_df) 
image_df

In [4]:
image_df = pd.DataFrame(image_df, columns=['specimen_no', 'Layer No.',
       'contours_threshold', 'area', 'long', 'short', 'aspect_ratio'])
image_df.dtypes

specimen_no             int64
Layer No.              object
contours_threshold    float64
area                   object
long                   object
short                  object
aspect_ratio           object
dtype: object

In [6]:
# image dataframe -> area 각 col 평균 값으로 치환한 df 생성
def make_area_df(image_df):
    '''
    시편 레이어별 이미지에서 contour 정보를 저장한 Image dataframe을 
    Input : 
    '''

    # object type -> float
    change_list = ['area', 'long', 'short', 'aspect_ratio']
    for i in change_list:
        image_df[i] = pd.to_numeric(image_df[i], errors='coerce')

    # 시편별 평균 값으로 치환 후 df화
    area_list = []
    item_list = set(image_df['specimen_no'].unique())
    for i in item_list:
        area = image_df[image_df['specimen_no'] == i]['area']
        long = image_df[image_df['specimen_no'] == i]['long']
        short = image_df[image_df['specimen_no'] == i]['short']
        aspect_ratio = image_df[image_df['specimen_no'] == i]['aspect_ratio']
        data = [i, round(area.mean(), 4), round(area.median(), 4), round(long.median(), 4), round(short.median(), 4), round(aspect_ratio.median(), 4)]
        area_list.append(data)
    columns = ['item_no', 'area_mean', 'area_median', 'long_median', 'short_median', 'aspect_ratio_median']
    area_df = pd.DataFrame(area_list, columns = columns)
    return area_df


In [ ]:
area_df = make_area_df(image_df)
area_df

In [ ]:
# 시편 메타데이터/Area 데이터 join
merge_data_2 = pd.merge(merge_data_to_corr,area_df, how='inner', on='item_no')
print('없는 시편 => ',set(merge_data_to_corr['item_no'].unique()) - set(area_df['item_no'].unique()))
# 1 ~ 48

In [ ]:
merge_data_2

#### 데이터 시각화

In [ ]:
sns.set_palette("colorblind")
sns.pairplot(merge_data_2, 
             palette="colorblind",
             vars=['temperature_mean', 'power(w)', 'speed(mm/sec)', 'energy_density','area_mean', 'long_median','area_median', 'short_median', 'aspect_ratio_median'] ,
                hue = 'specimen_position')

In [ ]:
merge_data_2_corr = pd.DataFrame(merge_data_2, columns= ['item_no', 'temperature_mean', 'temperature_median', 'power(w)',
       'speed(mm/sec)', 'specimen_position', 'energy_density', 'area_mean','area_median', 'long_median', 'short_median', 'aspect_ratio_median'])

draw_corrmap(merge_data_2_corr)

In [ ]:
scatterplot(x="temperature_mean", y="area_mean", data=merge_data_2_corr)

In [ ]:
scatterplot(x="temperature_mean", y="area_median", data=merge_data_2_corr)

In [ ]:
scatterplot(x="temperature_mean", y="long_median", data=merge_data_2_corr)

In [ ]:
scatterplot(x="temperature_mean", y="short_median", data=merge_data_2_corr)

In [ ]:
scatterplot(x="temperature_mean", y="aspect_ratio_median", data=merge_data_2_corr)

### 2) 45도
---
- DtypeWarning: Columns (4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(file_path)
[관련 Stakoverflow](https://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options)


In [ ]:
path = "C:\\Users\\Chaaaaabbo\\Desktop\\3DP\\230201_신규데이터진행\\"
files = os.listdir(path)

list_of_df = []
for file in os.listdir(path):
    file_path = os.path.join(path, file)
    df_temp = pd.read_csv(file_path, low_memory=False) 
                        #   ,dtype={'area':str, 'long':str, 'short':str, 'aspect_ratio' :str})
    list_of_df.append(df_temp) 

image_df = pd.concat(list_of_df) 
image_df

In [47]:
image_df = pd.DataFrame(image_df, columns=['specimen_no', 'Layer No.',
       'contours_threshold', 'area', 'long', 'short', 'aspect_ratio'])
image_df.dtypes

specimen_no             int64
Layer No.              object
contours_threshold    float64
area                   object
long                   object
short                  object
aspect_ratio           object
dtype: object

In [ ]:
area_df = make_area_df(image_df)
area_df

In [ ]:
merge_data_2 = pd.merge(merge_data_to_corr,area_df, how='inner', on='item_no')
print('없는 시편 => ',set(merge_data_to_corr['item_no'].unique()) - set(area_df['item_no'].unique()))

# 49 ~ 80

In [ ]:
merge_data_2

#### 데이터 시각화

In [ ]:
sns.set_palette("colorblind")
sns.pairplot(merge_data_2, 
             palette="colorblind",
             vars=['temperature_mean', 'power(w)', 'speed(mm/sec)', 'energy_density','area_mean', 'long_median','area_median', 'short_median', 'aspect_ratio_median'] ,
                hue = 'specimen_position')

In [ ]:
merge_data_2_corr = pd.DataFrame(merge_data_2, columns= ['item_no', 'temperature_mean', 'temperature_median', 'power(w)',
       'speed(mm/sec)', 'specimen_position', 'energy_density', 'area_mean','area_median', 'long_median', 'short_median', 'aspect_ratio_median'])

draw_corrmap(merge_data_2_corr)

In [ ]:
scatterplot(x="temperature_mean", y="area_median", data=merge_data_2_corr)

In [ ]:
scatterplot(x="temperature_mean", y="area_mean", data=merge_data_2_corr)

In [ ]:
scatterplot(x="temperature_mean", y="long_median", data=merge_data_2_corr)

In [ ]:
scatterplot(x="temperature_mean", y="short_median", data=merge_data_2_corr)

In [ ]:
scatterplot(x="temperature_mean", y="aspect_ratio_median", data=merge_data_2_corr)

### 3) 90도
---


In [ ]:
path = "C:\\Users\\Chaaaaabbo\\Desktop\\3DP\\230201_신규데이터진행\\"
files = os.listdir(path)

list_of_df = []
for file in os.listdir(path):
    file_path = os.path.join(path, file)
    df_temp = pd.read_csv(file_path)
    list_of_df.append(df_temp) 

image_df = pd.concat(list_of_df) 
image_df

In [ ]:
image_df = pd.DataFrame(image_df, columns=['specimen_no', 'Layer No.',
       'contours_threshold', 'area', 'long', 'short', 'aspect_ratio'])
image_df.dtypes

In [ ]:
area_df = make_area_df(image_df)
area_df

In [ ]:
merge_data_2 = pd.merge(merge_data_to_corr,area_df, how='inner', on='item_no')
print('없는 시편 => ',set(merge_data_to_corr['item_no'].unique()) - set(area_df['item_no'].unique()))

# 97 ~ 144

In [ ]:
merge_data_2

#### 데이터 시각화

In [ ]:
sns.set_palette("colorblind")
sns.pairplot(merge_data_2, 
             palette="colorblind",
             vars=['temperature_mean', 'power(w)', 'speed(mm/sec)', 'energy_density','long_median', 'short_median', 'aspect_ratio_median'] ,
                hue = 'specimen_position')

In [ ]:
merge_data_2_corr = pd.DataFrame(merge_data_2, columns= ['item_no', 'temperature_mean', 'temperature_median', 'power(w)',
       'speed(mm/sec)', 'specimen_position', 'energy_density', 'area_mean','area_median', 'long_median', 'short_median', 'aspect_ratio_median'])

draw_corrmap(merge_data_2_corr)

In [ ]:
scatterplot(x="temperature_mean", y="area_mean", data=merge_data_2_corr)

In [ ]:
scatterplot(x="temperature_mean", y="area_median", data=merge_data_2_corr)

In [ ]:
scatterplot(x="temperature_mean", y="long_median", data=merge_data_2_corr)

In [ ]:
scatterplot(x="temperature_mean", y="short_median", data=merge_data_2_corr)

In [ ]:
scatterplot(x="temperature_mean", y="aspect_ratio_median", data=merge_data_2_corr)